# Exploring World Historical Battle Database
Access to this database was granted by its creator and curator, [Dr. Shuhei Kitamura of Osaka University](https://osf.io/j357k). It's important we acknowledge his generosity. 

In the interest of looking at applying data science to social studies topics, Canada's involvement in world conflict is an important part of Canadian history. This database allows us to explore battles from throughout human history, and across the world.

![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)

<a href="https://hub.callysto.ca/jupyter/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fcallysto%2Fcurriculum-notebooks&branch=master&subPath=SocialStudies/HansardAnalysis/hansard-analysis.ipynb&depth=1" target="_parent"><img src="https://raw.githubusercontent.com/callysto/curriculum-notebooks/master/open-in-callysto-button.svg?sanitize=true" width="123" height="24" alt="Open in Callysto"/></a>

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import warnings
import pycountry_convert as pc
import requests

In [3]:
!pip install pycountry_convert

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 2.0 MB/s eta 0:00:0000:010:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.7/200.7 kB 2.8 MB/s eta 0:00:00a 0:00:01
  Created wheel for pycountry: filename=pycountry-22.3.5-py2.py3-none-any.whl size=10681832 sha256=84364a7d1a1e1f38f88aee1efe2a1e10ea97d201455d0585e502a6229d0a4c2c
  Stored in directory: /Users/marygrant/Library/Caches/pip/wheels/47/15/92/e6dc85fcb0686c82e1edbcfdf80cfe4808c058813fed0baa8f
Successfully built pycountry

[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [ ]:
# Read in the data
data = pd.read_excel('https://raw.githubusercontent.com/callysto/data-files/main/SocialStudies/HistoricalBattles/whbd_v11.xlsx')
data2 = pd.read_csv('https://raw.githubusercontent.com/callysto/data-files/main/SocialStudies/HistoricalBattles/historical_battles.csv')
data = data.merge(data2,how='left')
data = data[data['year'].notna()]
data['year'] = data['year'].map(lambda x: int(x))
data.sort_values('year',inplace=True)
data

In [ ]:
# See what the columns contain
data.columns

### CANADA

In [ ]:
# Filter to only look at the battles that specify Canada as a combatant. Drop unused columns
country_data = data[data['bell']=='Canada'].drop(['uk', 'fr', 'de', 'sp', 'sw', 'tr', 'at', 
                                             'ru', 'nl', 'it', 'pt', 'dk', 'habsburg', 'hre',
                                             'naval', 'river', 'lake', 'air', 'multiple', 'entire',
                                             'ongoing', 'plan', 'nopage', 'unknown'], axis=1)

In [ ]:
fig = px.scatter_geo(country_data, lat='lat', lon='lng', 
               hover_name='bname', 
               hover_data=['year'],
               color= 'war',
               title='Battles participated in by post-Confederation Canada')

fig.update_layout(showlegend=False)
fig.show()

In [ ]:
casualties_con = country_data[country_data['casualties'].notna()]

px.scatter_geo(casualties_con, lat='lat', lon='lng', 
               height=800, hover_name='bname', 
               hover_data=['war', 'year'],
               size = 'casualties',
               color='casualties',
               title='Casualties of battles participated in by Canada')

In [ ]:
con_bloodiest_war = country_data.groupby('war',as_index=False)['casualties'].sum()
con_bloodiest_war = con_bloodiest_war[con_bloodiest_war['casualties'] > 0]
con_bloodiest_war.sort_values('casualties',inplace=True)

In [ ]:
top_10_bloddiest_con_wars = con_bloodiest_war[:10:-1]

fig = px.bar(top_10_bloddiest_con_wars,x='war',y='casualties',title= "Canada's Bloodiest Wars")

fig.show()

In [ ]:
# Southernmost point of Canada is Lake Erie, ON, at 41°40' N; easternmost is Cape Spear, NL, at 52°37' W
# We also need to exclude a single WWII Pacific battle that happened off the coast of Alaska that didn't involve Canada
NA_data = data[(data['lat']>41.6) & 
               (data['lng']<-52.6) & 
               (data['lng']>-160)]

# There's also many wars in this subset that don't feature Canada, so we can list them here to remove:
remove = ['American Revolutionary War',
          'Sioux Wars',
          "Red Cloud's War",
          'Dakota War of 1862',
          'Russo-Tlingit War',
          'Great Sioux War of 1876',
          'Powder River Expedition',
          'American Civil War',
          'Yellowstone Expedition of 1873',
          'Nez Perce War',
          'Comanche Campaign',
          'Boston campaign',
          'Modoc War',
          'American Revolution',
          "King Philip's War",
          'Black Hawk War',
          'Colorado War',
          'American Indian Wars',
          'Forage War',
          "Coeur d'Alene War",
          'Yakima War',
          'Philadelphia campaign',
          'Ghost Dance War']

# Remove wars listed above, as well as battles without a specific war (that all happened in the USA)
NA_data = NA_data[(~NA_data['war'].isin(remove)) & (~NA_data['war'].isnull())]
          
          
          
fig = px.scatter_geo(NA_data, lat='lat', lon='lng', 
               hover_name='bname', 
               color='war',
               hover_data=['year'], 
               fitbounds='locations',
               title='Historical battles fought in present-day Canada')

fig.update_layout(showlegend=False)
fig.show()

In [ ]:
NA_casualties = NA_data[NA_data['casualties'].notna()]

px.scatter_geo(NA_casualties, lat='lat', lon='lng', 
               height=800, hover_name='bname', 
               size = 'casualties',
               size_max = 30,
               color = 'casualties',
               hover_data=['war','year'], 
               fitbounds='locations',
               title='Historical battles fought in present-day Canada')

In [ ]:
fig = px.scatter_mapbox(NA_data, lat="lat", lon="lng", hover_name="bname", hover_data=['year'],
                        color_discrete_sequence=["fuchsia"],zoom=2.5)
fig.update_layout(
    mapbox_style="white-bg",
    mapbox_layers=[
        {
            "below": 'traces',
            "sourcetype": "raster",
            "sourceattribution": "United States Geological Survey",
            "source": [
                "https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryOnly/MapServer/tile/{z}/{y}/{x}"
            ]
        }
      ])
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

### WORLD

In [ ]:
country_of_interest = 'Russia' #you can change this to a country you are interested in

# Filter to only look at the battles that specify Canada as a combatant. Drop unused columns
country_of_interest_data = data[data['bell']==country_of_interest].drop(['uk', 'fr', 'de', 'sp', 'sw', 'tr', 'at', 
                                             'ru', 'nl', 'it', 'pt', 'dk', 'habsburg', 'hre',
                                             'naval', 'river', 'lake', 'air', 'multiple', 'entire',
                                             'ongoing', 'plan', 'nopage', 'unknown'], axis=1)
display(country_of_interest_data)

We can plot the battles on a map with their lattitude and longitude coordinates.

You can drag to move around the map, zoom in and out to get more clarity. Hovering over each data point lists the name of the battle, as well as the war in which the battle was fought.

In [ ]:
country_of_interest_data = country_of_interest_data[country_of_interest_data['bname'].notna()]

fig = px.scatter_geo(country_of_interest_data, lat='lat', lon='lng', 
               hover_name='bname', 
               hover_data=['year'],
               color= 'war',
               title='Battles participated in by ' + country_of_interest)

fig.update_layout(showlegend=False)
fig.show()

Let's animate through the years so see the wars that Canada has participated in

In [ ]:
warnings.filterwarnings("ignore")
years = list(country_of_interest_data['year'].unique())

animate_country_data = pd.DataFrame(country_of_interest_data)

def animation_years(row):
    global animate_country_data
   
    df = pd.DataFrame(columns=animate_country_data.columns)
    index = years.index(row['year'])
    for i in years[index+1:]:
        row[4] = i
        df.loc[len(df.index)] = row
    
    animate_country_data = pd.concat([animate_country_data,df],ignore_index=True)

for i in range(len(country_of_interest_data.index)):
    animation_years(country_of_interest_data.iloc[i])

animate_country_data

In [ ]:
def rank_rows(df):
    
    df['sort_rank'] = 0
    curr_rank = 1
    for b in battles:
        temp = df.loc[df['bname'] == b]
        for i,rows in temp.iterrows():  
            if df['sort_rank'][i] == 0:
                df['sort_rank'][i] = curr_rank
                curr_rank += 1
    return df        
            

    
animate_country_data.sort_values(['year'],inplace=True)            
battles = list(animate_country_data.sort_values('year')['bname'].unique())
animate_country_data = rank_rows(animate_country_data)

animate_country_data.sort_values('sort_rank',inplace=True)
animate_country_data

By clicking on the "play" button on the bottom we'll be able to look at the wars that Canada has found throughout the years

In [ ]:
px.scatter_geo(animate_country_data, lat='lat', lon='lng', 
               height=800, hover_name='bname', 
               animation_frame= 'year',
               animation_group='war',
               title='Battles participated in by ' + country_of_interest)

We can also look at which continents Canada has fought the most in.

In [ ]:
countries = ['Canada','Japan','India','Spain'] #you can change and add/remove countries in this list

battles_continent = data
battles_continent = battles_continent[battles_continent['continent'].notna()]
battles_continent['interest'] = data['bell'].map(lambda x: True if x in countries else False)
battles_continent = battles_continent[battles_continent['interest'] == True]
battles_continent.rename(columns={'bell':'Country'},inplace=True)
battles_continent

In [ ]:
continent_grouped = battles_continent.groupby(['continent','Country'])['locn'].count().reset_index(name='Number of Battles Fought')
continent_grouped.sort_values('Number of Battles Fought',ascending=False,inplace=True)
fig = px.bar(continent_grouped,x='Country',y='Number of Battles Fought',color='continent',title='Number of Battles fought by each country in different Continents')
fig.show()